## Extract

Extract the data

In [17]:
import boto3
import pandas as pd
from io import StringIO

# create s3 client
s3 = boto3.client('s3')

# specify the bucket & folder you'd like to interact with
# will be using the cyber data as an example
name = "data-analyst-job-east"
file = "raw/gsearch_jobs.csv"

single_object = s3.get_object(Bucket=name, Key=file)
single_df = pd.read_csv(single_object['Body'])

In [18]:
single_df.head()

,Unnamed: 0,index,title,company_name,location,via,description,extensions,job_id,thumbnail,...,commute_time,salary_pay,salary_rate,salary_avg,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens
0,0,0,Data Analyst,Chloeta,"Oklahoma City, OK",via ZipRecruiter,Job Summary: The Data Analyst oversees data pr...,"['21 hours ago', 'Full-time', 'Health insuranc...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['python', 'r']"
1,1,1,Junior Data Analyst/Scientist Role - Contract ...,Upwork,Anywhere,via Upwork,"Company\n\nThe TAC Index provides independent,...","['17 hours ago', 'Work from home', 'Contractor...",eyJqb2JfdGl0bGUiOiJKdW5pb3IgRGF0YSBBbmFseXN0L1...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['matlab', 'matplotlib', 'postgresql', 'mongod..."
2,2,2,Data Analyst,ATC,United States,via LinkedIn,Job Title: Entry Level Business Analyst / Prod...,"['12 hours ago', 'Full-time', 'Health insurance']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
3,3,3,Data Analyst,Guidehouse,"Topeka, KS",via Nexxt,Job Family :\n\nData Science & Analysis (Digit...,"['10 hours ago', 'Full-time', 'Health insuranc...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['powerpoint', 'perl', 'python', 'qlik', 'tabl..."
4,4,4,Data Analyst,AnMed Health LLC,Anywhere,via LinkedIn,"AnMed is a dynamic, comprehensive health syste...","['18 hours ago', 'Work from home', 'Part-time'...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [19]:
single_df['company_name'].value_counts()


company_name
Upwork                             6934
Talentify.io                       1683
Walmart                            1537
EDWARD JONES                        747
Dice                                691
                                   ... 
Fisher Financial  Advisors, LLC       1
e-Emphasys Technologies               1
The Marlin Alliance, Inc.             1
Climate People                        1
Techdash Telecom                      1
Name: count, Length: 11019, dtype: int64

## Transform

In [20]:
# 1) Drop staffing agencies (keywords to Staffing, Hire, Global, etc)

filtered_df = single_df[~single_df['company_name'].isin(['Upwork', 'Talentify.io'])]

company_counts = filtered_df['company_name'].value_counts()


In [21]:
filtered_df['company_name'].value_counts()

company_name
Walmart               1537
EDWARD JONES           747
Dice                   691
Corporate              612
Cox Communications     538
                      ... 
IDme                     1
Teads                    1
Jkbarnes                 1
New Globe Inc            1
Techdash Telecom         1
Name: count, Length: 11017, dtype: int64

In [22]:
filtered_df.columns

Index(['Unnamed: 0', 'index', 'title', 'company_name', 'location', 'via',
       'description', 'extensions', 'job_id', 'thumbnail', 'posted_at',
       'schedule_type', 'work_from_home', 'salary', 'search_term', 'date_time',
       'search_location', 'commute_time', 'salary_pay', 'salary_rate',
       'salary_avg', 'salary_min', 'salary_max', 'salary_hourly',
       'salary_yearly', 'salary_standardized', 'description_tokens'],
      dtype='object')

In [23]:
# 2) Most important columns: description_token, salary_standardized, title, date_time   (drop everything else)

selected_columns_df = filtered_df[['description_tokens', 'salary_standardized', 'title', 'date_time']]
selected_columns_df

,description_tokens,salary_standardized,title,date_time
0,"['python', 'r']",NaN,Data Analyst,2023-08-03 03:00:09.849838
2,[],NaN,Data Analyst,2023-08-03 03:00:09.849838
3,"['powerpoint', 'perl', 'python', 'qlik', 'tabl...",NaN,Data Analyst,2023-08-03 03:00:09.849838
4,[],NaN,Data Analyst,2023-08-03 03:00:09.849838
5,[],NaN,Research Data Analyst 1,2023-08-03 03:00:09.849838
...,...,...,...,...
48392,"['python', 'sql', 'tableau', 'snowflake', 'r',...",103781.0,Marketing Data & BI Analyst II,2022-11-04 03:40:23.706734
48393,[],144481.5,Lead-Data Analyst,2022-11-24 04:00:08.710801
48394,[],144481.5,Lead-Data Analyst,2022-12-07 04:00:12.563831
48395,[],144481.5,Lead-Data Analyst,2022-12-08 04:00:15.975728


In [24]:
# 3) drop 39,000 missing rows of salary_standardized

selected_columns_df.isnull().sum() 


description_tokens         0
salary_standardized    34845
title                      0
date_time                  0
dtype: int64

In [25]:
selected_columns_df = selected_columns_df.dropna(subset=['salary_standardized'])
selected_columns_df

,description_tokens,salary_standardized,title,date_time
19,"['pl/sql', 'sql', 'tableau']",94640.0,"Data Analyst Report Writer (Level 2) _ Austin,...",2023-08-03 03:00:11.064921
29,[],48391.2,Associate Research/Data Analyst-CES - Now Hiring,2023-08-03 03:00:13.367401
96,"['python', 'sql', 'r']",86320.0,Bioinformatics Analyst (NGS/OMICS/SQL/Curation...,2023-08-03 03:01:00.649092
98,"['python', 'sql', 'r', 'tableau']",122000.0,Data Analyst,2023-08-04 03:00:13.797776
102,"['powerpoint', 'word', 'excel', 'outlook']",100000.0,Data Analyst | Workforce Management,2023-08-04 03:00:13.797776
...,...,...,...,...
48392,"['python', 'sql', 'tableau', 'snowflake', 'r',...",103781.0,Marketing Data & BI Analyst II,2022-11-04 03:40:23.706734
48393,[],144481.5,Lead-Data Analyst,2022-11-24 04:00:08.710801
48394,[],144481.5,Lead-Data Analyst,2022-12-07 04:00:12.563831
48395,[],144481.5,Lead-Data Analyst,2022-12-08 04:00:15.975728


In [ ]:
# 4) join in the missing data


In [26]:
import boto3
import pandas as pd
from io import StringIO

# create s3 client
s3 = boto3.client('s3')

# specify the bucket & folder you'd like to interact with
# will be using the cyber data as an example
name = "data-analyst-job-east"
file = "raw/LinkedIn_job_list.csv"

Second_object = s3.get_object(Bucket=name, Key=file)
Second_df = pd.read_csv(Second_object['Body'])

In [27]:
Second_df.head()

,job_title,company_name,time_posted,num_applicants,employment_type,salary
0,"Data Scientist, Small Business Group",Meta,1 day ago,77 applicants,Not Applicable,"Base pay range\r\n\r\n $134,000.00/yr - $..."
1,Junior Data Scientist,Team Remotely Inc,1 day ago,100 applicants,Entry level,NaN
2,ML Engineer-AI/ML,Zortech Solutions,1 day ago,49 applicants,Entry level,NaN
3,"Data Scientist, Analytics",DoorDash,NaN,NaN,Entry level,"Base pay range\r\n\r\n $133,000.00/yr - $..."
4,Junior Data Scientist,LTIMindtree,NaN,NaN,Entry level,NaN


In [28]:
Second_df = Second_df.rename(columns={'job_title': 'title'})

In [29]:
import re

In [31]:

def extract_average_salary(salary_range):
    if isinstance(salary_range, str):
        salaries = re.findall(r'\$\d{1,3}(?:,\d{3})*(?:\.\d{2})?', salary_range)
        salaries = [float(s.replace('$', '').replace(',', '')) for s in salaries]
        if salaries:
            return sum(salaries) / len(salaries)
    return None
Second_df['average_salary'] = Second_df['salary'].apply(extract_average_salary)
Second_df = Second_df.dropna(subset=['average_salary'])
overall_average_salary = Second_df['average_salary'].mean()


In [32]:
Second_df

,title,company_name,time_posted,num_applicants,employment_type,salary,average_salary
0,"Data Scientist, Small Business Group",Meta,1 day ago,77 applicants,Not Applicable,"Base pay range\r\n\r\n $134,000.00/yr - $...",169000.0
3,"Data Scientist, Analytics",DoorDash,NaN,NaN,Entry level,"Base pay range\r\n\r\n $133,000.00/yr - $...",188000.0
5,"Data Scientist, Analytics",DoorDash,NaN,NaN,Entry level,"Base pay range\r\n\r\n $133,000.00/yr - $...",188000.0
7,Jr Data Scientist,EVONA,NaN,NaN,Entry level,"Base pay range\r\n\r\n $100,000.00/yr - $...",110000.0
9,"Data Scientist, Analytics",DoorDash,NaN,NaN,Entry level,"Base pay range\r\n\r\n $133,000.00/yr - $...",188000.0
...,...,...,...,...,...,...,...
333,Business Analyst,Insight Global,NaN,149 applicants,Mid-Senior level,Base pay range\r\n\r\n $38.00/hr - $50.00/hr,44.0
335,Senior Data Scientist,"Qventus, Inc",NaN,111 applicants,Mid-Senior level,"Base pay range\r\n\r\n $170,000.00/yr - $...",180000.0
338,"Staff, Data Scientist",Walmart,5 days ago,NaN,Entry level,"Base pay range\r\n\r\n $143,000.00/yr - $...",214500.0
344,Business Systems Analyst,Vernovis,NaN,78 applicants,Mid-Senior level,"Base pay range\r\n\r\n $110,000.00/yr - $...",112500.0


In [33]:
# Combine the dataframes
combined_df = pd.concat([single_df, Second_df], ignore_index=True)


In [34]:
combined_df

,Unnamed: 0,index,title,company_name,location,via,description,extensions,job_id,thumbnail,...,salary_min,salary_max,salary_hourly,salary_yearly,salary_standardized,description_tokens,time_posted,num_applicants,employment_type,average_salary
0,0.0,0.0,Data Analyst,Chloeta,"Oklahoma City, OK",via ZipRecruiter,Job Summary: The Data Analyst oversees data pr...,"['21 hours ago', 'Full-time', 'Health insuranc...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,...,NaN,NaN,NaN,NaN,NaN,"['python', 'r']",NaN,NaN,NaN,NaN
1,1.0,1.0,Junior Data Analyst/Scientist Role - Contract ...,Upwork,Anywhere,via Upwork,"Company\n\nThe TAC Index provides independent,...","['17 hours ago', 'Work from home', 'Contractor...",eyJqb2JfdGl0bGUiOiJKdW5pb3IgRGF0YSBBbmFseXN0L1...,NaN,...,NaN,NaN,NaN,NaN,NaN,"['matlab', 'matplotlib', 'postgresql', 'mongod...",NaN,NaN,NaN,NaN
2,2.0,2.0,Data Analyst,ATC,United States,via LinkedIn,Job Title: Entry Level Business Analyst / Prod...,"['12 hours ago', 'Full-time', 'Health insurance']",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,3.0,3.0,Data Analyst,Guidehouse,"Topeka, KS",via Nexxt,Job Family :\n\nData Science & Analysis (Digit...,"['10 hours ago', 'Full-time', 'Health insuranc...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,https://encrypted-tbn0.gstatic.com/images?q=tb...,...,NaN,NaN,NaN,NaN,NaN,"['powerpoint', 'perl', 'python', 'qlik', 'tabl...",NaN,NaN,NaN,NaN
4,4.0,4.0,Data Analyst,AnMed Health LLC,Anywhere,via LinkedIn,"AnMed is a dynamic, comprehensive health syste...","['18 hours ago', 'Work from home', 'Part-time'...",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48501,NaN,NaN,Business Analyst,Insight Global,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149 applicants,Mid-Senior level,44.0
48502,NaN,NaN,Senior Data Scientist,"Qventus, Inc",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111 applicants,Mid-Senior level,180000.0
48503,NaN,NaN,"Staff, Data Scientist",Walmart,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5 days ago,NaN,Entry level,214500.0
48504,NaN,NaN,Business Systems Analyst,Vernovis,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78 applicants,Mid-Senior level,112500.0


## Load

In [39]:
# 5) Load data back into s3 bucket
import boto3

# open client
client = boto3.client('s3')

# some initial variables
bucket_name = "test-ds2024-bucket"

# open the file in binary format, and save into the var 'data'
with open("data/Tux.svg.png", "rb") as f:
    data = f.read()

### ADDING OBJECTS TO A BUCKET ###
response = client.put_object(
    Body=data,
    Bucket=bucket_name,
    Key="tux-image"
)

FileNotFoundError: [Errno 2] No such file or directory: 'data/Tux.svg.png'